In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

In [9]:
with open('vicRoads_faq.json','rt') as f:
    raw_documents = json.load(f)

In [2]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
index = faiss.read_index("vr_faq_index.idx")

In [11]:
query = "I am from USA can I drive with my licence"
query_embedding = embedding_model.encode(query)
query_embedding = np.array(query_embedding).astype('float32').reshape(1,-1)

In [12]:
k = 5  # Number of results to retrieve
distances, indices = index.search(query_embedding, k)

In [13]:
for i, idx in enumerate(indices[0]):
    if idx < len(raw_documents):  # Ensure the index is within bounds
        print(f"Result {i + 1}:")
        print(f"Section: {raw_documents[idx]['Section']}")
        print(f"Question: {raw_documents[idx]['Question']}")
        print(f"Answer: {raw_documents[idx]['Answer']}")
        print(f"Distance: {distances[0][i]}\n")
    else:
        print(f"Result {i + 1}: No corresponding FAQ found for index {idx}")

Result 1:
Section: Learner permit
Question: What is a learner permit?
Answer: When you get your learner permit, you can drive a car on the road with a supervising driver.
To get your learner permit, you need to pass the Learner Permit Test.

Distance: 0.963636577129364

Result 2:
Section: Converting licence
Question: Can I drive in Victoria with my overseas licence?
Answer: You can drive in Victoria with your overseas licence if:

Your licence is current and valid
Your licence covers the type of vehicle you are driving
You have been living in Victoria for less than 6 months in total.
Distance: 1.0315978527069092

Result 3:
Section: Converting licence
Question: What tests might I need to take to convert my overseas licence?
Answer: You might need to take the following tests:

Road Law Knowledge Test (Learner Permit Test)
Hazard Perception Test
Drive Test.
The Learner Permit Test and Hazard Perception Test can be taken online or in person, while the Drive Test must be done in person.
Dis